# Multi-effect distillation (MED) model validation

TODO: Use pre-processing and data conditioning functions from `models_psa.utils`


In [44]:
from pathlib import Path
import hjson
import numpy as np
import pandas as pd
import time
from loguru import logger
import json

import MED_model
import matlab

from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot

from solarmed_modeling.utils import data_preprocessing, data_conditioning
from solarmed_modeling.data_validation import within_range_or_nan_or_max, within_range_or_zero_or_max

%load_ext autoreload
%autoreload 2

logger.disable("phd_visualizations.utils.units")

resample_figures = False

# Paths
data_path: Path = Path("../../data")

    
MED_model_matlab = MED_model.initialize()
logger.info('MATLAB engine initialized')


date_str: str = 20230511 # "20230511"  # '20230630'
sample_rate: int = 400


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2025-01-04 08:34:54.671 | INFO     | __main__:<module>:31 - MATLAB engine initialized


In [45]:
# Load data
with open(data_path / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)

# Load data and preprocess data
df = data_preprocessing([data_path / f"datasets/{fn}" for fn in [f"{date_str}_solarMED.csv", f"{date_str}_MED.csv"]], 
                        vars_config, sample_rate_key=f"{sample_rate}s")
# Condition data
df = data_conditioning(df, sample_rate_numeric=sample_rate, vars_config=vars_config,)

df


/workspaces/solarMED-modeling/src/solarmed_modeling/utils/__init__.py:138: DtypeWarning:

Columns (42,50,66,73,75) have mixed types. Specify dtype option on import or set low_memory=False.

/workspaces/solarMED-modeling/src/solarmed_modeling/utils/__init__.py:167: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

2025-01-04 08:34:55.499 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:81 - Signal id PK-MED-E03-pa not found in dataframe columns.
2025-01-04 08:34:55.501 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:81 - Signal id FT-SF-003 not found in dataframe columns.


,qmed_s,qts_dis_original,Tts_h_out,Tmed_s_in,Tts_c_in,ZT-AQU-TCV102,wmed_f,qmed_f,qmed_d,Tmed_c_out,...,Psf_l2,Psf_l3,Psf_l4,Psf_l5,Pth_ts_src,Pth_hx_s,Pth_ts_dis,sf_active,med_active,ts_active
time,,,,,,,,,,,,,,,,,,,,,
2023-05-11 07:26:40+00:00,0.0,0.033446,75.749327,49.095691,43.006526,0.593134,103.759155,0.0,0.0,18.195833,...,1.236760,0.672183,3.570127,2.223859,0.0,0.0,0.129382,True,False,False
2023-05-11 07:33:20+00:00,0.0,0.033472,75.527441,49.057274,42.932932,0.599137,103.760875,0.0,0.0,18.318358,...,3.338351,3.068519,6.019116,4.634292,0.0,0.0,0.129298,True,False,False
2023-05-11 07:40:00+00:00,0.0,0.033436,75.516380,48.817047,42.824880,0.604186,103.762608,0.0,0.0,18.494865,...,3.783896,3.747119,7.010392,5.573010,0.0,0.0,0.126668,True,False,False
2023-05-11 07:46:40+00:00,0.0,0.033453,75.353377,48.603423,42.722086,0.605678,103.764347,0.0,0.0,18.684251,...,3.011230,3.189443,6.601975,4.692048,0.0,0.0,0.123942,True,False,False
2023-05-11 07:53:20+00:00,0.0,0.033479,75.421423,48.419229,42.629829,0.608455,103.765625,0.0,0.0,18.801903,...,2.997304,2.993600,6.414546,4.199520,0.0,0.0,0.122860,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-11 16:46:40+00:00,0.0,0.032713,48.805451,49.033724,55.275760,99.971269,8.061619,0.0,0.0,26.493845,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-0.000000,True,False,True
2023-05-11 16:53:20+00:00,0.0,0.032705,48.480917,48.964636,54.926018,99.967698,8.249758,0.0,0.0,26.359626,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-0.000000,True,False,True
2023-05-11 17:00:00+00:00,0.0,0.032902,48.183382,48.857430,54.576655,99.965503,8.213361,0.0,0.0,26.233246,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-0.000000,True,False,True


In [1]:
# plt_config = {
#   # General plot attributes
#   "title": "MED",
#   "subtitle": "Model validation",
#   "height": 1200,
#   "width": 1000,
#   "margin": {
#     "l":20,
#     "r":100,
#     "t":100,
#     "b":20,
#     "pad":5
#   },
#   "vertical_spacing": 0.03,
#   "xdomain": [0, 0.85],
#   "arrow_xrel_pos": 60, # seconds

#   "plots": {
#     "med_flows": {
#       "title": "<b>MED</b>",
#       "row_height": 1,
#       "bg_color": "bg_gray",
#       "ylabels_left": ["m<sup>3</sup>/h"],
#       "ylabels_right": ["m<sup>3</sup>/h"],
#       "tigth_vertical_spacing": True,
#       "ylims_left": 'manual',

#       # Plotted between this plot and the next one
#       "show_active": True,
#       "active_var_id": "med_active",
#       "active_color": "dc_green_rgb", # Needs to end with _rgb

#       "traces_left": [
#         {
#           "var_id": "qmed_f",
#           "color": "c_blue",
#           "mode": "lines",
#           "dash": "dot"
#         },
#         {
#           "var_id": "qmed_d",
#           "color": "plotly_green",
#           "mode": "lines",
#           "dash": "dashdot"
#         },
#         {
#           "var_id": "qmed_c",
#           "color": "plotly_cyan",
#           "mode": "lines",
#           "dash": "dash",
#         }
#       ],

#       "traces_right": [
#         {
#           "var_id": "qmed_s",
#           "mode": "lines",
#           "color": "plotly_red",
#           "axis_arrow": True,
#           "arrow_yrel_pos": 0.9,
#         },
#       ]
#     },

#     "med_temperatures": {
#       "row_height": 1,
#       "bg_color": "bg_gray", # bg gray
#       "ylabels_left": ["⁰C"],
#       "ylims_left": 'manual',

#       "traces_left": [
#         {
#           "var_id": "Tmed_s_in",
#           "mode": "lines",
#           "color": "plotly_red",
#         },
#         {
#           "var_id": "Tmed_s_out",
#           "mode": "lines",
#           "color": "plotly_red",
#           "dash": "dash",
#         },
#         {
#           "var_id": "Tmed_c_in",
#           "mode": "lines",
#           "color": "c_blue",
#         },
#         {
#           "var_id": "Tmed_c_out",
#           "mode": "lines",
#           "color": "c_blue",
#           "dash": "dash"
#         },
#       ]
#     },

#   }

# }


In [9]:
# # Save to json
# import json
# 
# with open( Path("../data") / 'plt_config_med.json', 'w') as f:
#     json.dump(plt_config, f, indent=4)


## Validate model training

In [18]:
# Visualize model fit
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Constants
input_ids: list[str] = ["Ms", "Ts_in", "Mf", "Tc_out", "Tc_in"]
output_ids: list[str] = ["Md", "Ts_out", "Mc"]
seed: int = 42

# Load data
with open(data_path / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)

data: pd.DataFrame = pd.read_csv(data_path / 'datasets/operation_points_v2.csv')

display(data)
print(f"{len(data)}")
print(data.columns)


,time,Ms,Mf,Md,Mc,Ts_in,Ts_out,Tf,Td,Pvc,...,Pv_1,Pv_2,Pv_4,Pv_6,Pv_8,Pv_10,Pv_12,Pv_14,Tph_ref_14,wf
0,2009-05-15 08:51:44+00:00,43.989517,7.969840,2.250647,8.279807,62.513095,59.544819,57.397665,30.493447,42.446557,...,179.143093,163.495657,120.529744,99.303712,80.946400,67.065822,55.930462,44.766894,29.646541,3.30624
1,2009-05-15 09:01:44+00:00,43.990298,7.971598,2.236411,8.287988,62.290946,59.334985,57.284245,30.971941,43.394932,...,178.138166,162.795273,120.845351,100.001232,81.883529,68.147188,57.032368,45.887061,30.147909,3.30624
2,2009-05-15 09:11:44+00:00,43.989575,7.965735,2.227206,8.285425,62.581557,59.636178,57.572030,31.477519,44.579493,...,180.292725,164.509903,122.296990,101.343218,83.129892,69.374153,58.350215,47.237415,30.738334,3.30624
3,2009-05-15 09:42:52+00:00,43.986815,7.953573,2.124227,8.282549,62.680053,59.859261,57.785068,32.788266,48.240648,...,182.199579,166.838977,125.356998,104.639433,86.616237,73.060534,62.122525,51.196886,32.280036,3.30624
4,2009-05-15 09:52:52+00:00,43.982423,7.947083,2.124882,8.279844,62.885613,60.067465,58.001170,33.294130,49.769733,...,184.086935,169.043699,127.253750,106.463384,88.332202,74.731545,63.821651,52.837378,32.876461,3.30624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,2012-04-17 10:47:45+00:00,41.106490,7.787207,1.221267,13.081017,71.829864,67.794467,64.336511,21.368971,44.359224,...,260.556974,229.314313,151.555567,121.170949,95.896905,77.206217,60.501202,45.130574,29.528484,3.30624
841,2012-04-17 10:55:45+00:00,41.104306,7.779161,1.185197,14.131246,71.698753,67.706546,64.280832,21.524272,44.407043,...,259.704036,228.774104,151.542921,121.234136,95.983937,77.289469,60.542123,45.175485,29.530594,3.30624
842,2012-04-17 11:03:45+00:00,41.107775,7.775968,1.201441,15.624742,71.801699,67.759812,64.263024,22.195718,44.246575,...,260.430011,229.121829,151.671249,121.274380,96.003379,77.269748,60.455831,45.009730,29.420515,3.30624
843,2012-04-17 11:11:45+00:00,41.104519,7.766992,1.226294,17.038622,71.894362,67.896687,64.357044,22.605675,44.381693,...,261.433354,229.991126,152.021154,121.638535,96.245695,77.451516,60.582871,45.148548,29.430709,3.30624


845
Index(['time', 'Ms', 'Mf', 'Md', 'Mc', 'Ts_in', 'Ts_out', 'Tf', 'Td', 'Pvc',
       'Tc_in', 'Tc_out', 'Pv_1', 'Pv_2', 'Pv_4', 'Pv_6', 'Pv_8', 'Pv_10',
       'Pv_12', 'Pv_14', 'Tph_ref_14', 'wf'],
      dtype='object')


In [39]:
# Filter operation points
"""
Points are considered different and thus valid if:
- ΔTs,in > 0.5 or
- ΔTs,out > 0.5 or
- ΔTvc > 1.5 or
- ΔMs > 0.2*3.6
- ΔMf > 0.5
- ΔTc,out > 0.5
"""

init_len = 0
filtered_data = data.copy()
while init_len != len(filtered_data):
    init_len = len(filtered_data)
    
    # Filter similar points based on the given conditions
    filtered_data = data[
        (data['Ts_in'].diff().abs() > 0.5) |
        (data['Ts_out'].diff().abs() > 0.5) |
        (data['Tc_out'].diff().abs() > 1.5) |
        (data['Ms'].diff().abs() / 3.6 > 0.2) |
        (data['Mf'].diff().abs() > 0.5) |
        (data['Tc_out'].diff().abs() > 0.5)
    ]
    
    # Filter points out of range
    filtered_data = data[
        (data['Ts_in'].between(52, 75)) &
        (data['Tc_out'].between(18, 40)) &
        (data['Tc_in'].between(12, 33.8)) &
        (data['Md'] >= 1.8)
    ]
    

display(filtered_data)
print(f"{len(filtered_data)}")

data = filtered_data

# Divide data between calibration / training and validation
X = data[input_ids].values
Y = data[output_ids].values

# Shuffle the indices
indices = np.arange(len(X))
np.random.seed(seed)  # For reproducibility
np.random.shuffle(indices)

# Define split point
split_index = int(0.8 * len(X))  # 80% for training, 20% for validation

# Split data
train_indices = indices[:split_index]
val_indices = indices[split_index:]

X_train, Y_train = X[train_indices], Y[train_indices]
X_val, Y_val = X[val_indices], Y[val_indices]

print(f"{X.shape=}, {Y.shape=}")
print(f"{Y_train.shape=}")
print(f"{Y_val.shape=}")



,time,Ms,Mf,Md,Mc,Ts_in,Ts_out,Tf,Td,Pvc,...,Pv_1,Pv_2,Pv_4,Pv_6,Pv_8,Pv_10,Pv_12,Pv_14,Tph_ref_14,wf
0,2009-05-15 08:51:44+00:00,43.989517,7.969840,2.250647,8.279807,62.513095,59.544819,57.397665,30.493447,42.446557,...,179.143093,163.495657,120.529744,99.303712,80.946400,67.065822,55.930462,44.766894,29.646541,3.30624
1,2009-05-15 09:01:44+00:00,43.990298,7.971598,2.236411,8.287988,62.290946,59.334985,57.284245,30.971941,43.394932,...,178.138166,162.795273,120.845351,100.001232,81.883529,68.147188,57.032368,45.887061,30.147909,3.30624
2,2009-05-15 09:11:44+00:00,43.989575,7.965735,2.227206,8.285425,62.581557,59.636178,57.572030,31.477519,44.579493,...,180.292725,164.509903,122.296990,101.343218,83.129892,69.374153,58.350215,47.237415,30.738334,3.30624
3,2009-05-15 09:42:52+00:00,43.986815,7.953573,2.124227,8.282549,62.680053,59.859261,57.785068,32.788266,48.240648,...,182.199579,166.838977,125.356998,104.639433,86.616237,73.060534,62.122525,51.196886,32.280036,3.30624
4,2009-05-15 09:52:52+00:00,43.982423,7.947083,2.124882,8.279844,62.885613,60.067465,58.001170,33.294130,49.769733,...,184.086935,169.043699,127.253750,106.463384,88.332202,74.731545,63.821651,52.837378,32.876461,3.30624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,2011-04-20 10:38:07+00:00,25.891146,8.054595,2.151164,12.729419,64.985897,59.928805,57.778854,29.072973,41.612805,...,183.109206,169.437085,124.743173,102.601095,82.401592,68.075981,54.778751,42.248786,28.606810,3.30624
581,2011-04-26 10:51:07+00:00,25.858430,6.018845,2.085503,8.806182,64.881452,59.956741,57.867972,28.653212,41.576920,...,181.420278,168.891486,123.194795,100.732394,81.360940,67.316296,54.335470,42.227161,28.702012,3.30624
582,2011-04-26 11:01:07+00:00,25.852777,6.012166,2.104099,9.668030,64.894876,59.961908,57.977237,29.030250,41.580232,...,182.536682,169.647905,124.602333,102.100709,82.491204,68.193053,54.777165,42.220583,28.667232,3.30624
583,2011-04-26 11:11:07+00:00,25.820407,6.007560,2.103695,10.414113,64.638266,59.876821,57.877583,29.206987,41.586318,...,181.847411,168.846433,124.474040,102.208168,82.612324,68.324115,54.869832,42.208215,28.651897,3.30624


480
X.shape=(480, 5), Y.shape=(480, 3)
Y_train.shape=(384, 3)
Y_val.shape=(96, 3)


In [40]:
# MsIn = matlab.double([data['Ms'].values / 3.6], size=(1, len(data)))  # m³/h -> L/s
# TsinIn = matlab.double([data['Ts_in'].values], size=(1, len(data)))
# MfIn = matlab.double([data['Mf'].values], size=(1, len(data)))
# TcwinIn = matlab.double([data['Tc_in'].values], size=(1, len(data)))
# op_timeIn = matlab.double([np.zeros((1, len(data)))], size=(1, len(data)))
# TcwoutIn = matlab.double([data['Tc_out'].values], size=(1, len(data)))

Y_preds = np.empty(Y_val.shape, dtype=float)
for i, idx in enumerate(val_indices):
    ds = data.iloc[idx]
    
    mmed_d, Tmed_s_out, mmed_c, _, _ = MED_model_matlab.MED_model(
        matlab.double([ds['Ms']/ 3.6], size=(1, 1)),  # L/s
        matlab.double([ds['Ts_in']], size=(1, 1)),  # ºC
        matlab.double([ds['Mf']], size=(1, 1)), # m³/h
        matlab.double([ds['Tc_out']], size=(1, 1)), # ºC
        matlab.double([ds['Tc_in']], size=(1, 1)), # ºC
        matlab.double([[0]], size=(1, 1)),  # hours
        nargout=5,
    )
    Y_preds[i, :] = mmed_d, Tmed_s_out, mmed_c


In [50]:
# Visualize fit

# Create traces for the plot
fig = make_subplots(rows=len(output_ids), cols=1, shared_xaxes=True, 
                    subplot_titles=output_ids, x_title="Data Index",
                    vertical_spacing=0.05)

for idx, output_id in enumerate(output_ids):
    # Add true values as a scatter plot
    fig.add_trace(go.Scatter(
        x=np.arange(Y_val.shape[0]),
        y=Y_val[:, idx],
        mode='markers',
        name='True values',
        showlegend=True if idx==0 else False,
        marker=dict(color='blue')
    ), row=idx+1, col=1)

    # Add predicted values as a line plot
    fig.add_trace(go.Scatter(
        x=np.arange(Y_preds[:, idx].shape[0]),
        y=Y_preds[:, idx],
        mode='lines',
        name='Predicted values',
        showlegend=True if idx==0 else False,
        line=dict(color='red')
    ), row=idx+1, col=1)

    # Add confidence interval as a shaded area
    # fig.add_trace(go.Scatter(
    #     x=np.concatenate([np.arange(Y_pred.shape[0]), np.arange(Y_pred.shape[0])[::-1]]),
    #     y=np.concatenate([Y_preds[:, idx] - 2 * np.sqrt(Y_pred_vars[:, idx]), 
    #                     (Y_preds[:, idx] + 2 * np.sqrt(Y_pred_vars[:, idx]))[::-1]]),
    #     fill='toself',
    #     fillcolor='rgba(255, 0, 0, 0.2)',
    #     line=dict(color='rgba(255, 0, 0, 0)'),
    #     name='Confidence interval (95%)',
    #     showlegend=True if idx==0 else False,
    # ), row=idx+1, col=1)

# Update layout
fig.update_layout(
    title="Regression Results",
    legend=dict(orientation="h", yanchor="bottom", y=1.1, xanchor="right", x=1),
    template="plotly_white",
    height=600,
    margin=dict(t=100)
)

# Show the plot
fig.show()

# Visualization 2. 
# Create a figure
fig = go.Figure()

# Iterate over each output variable
for i, output_name in enumerate(output_ids):
    # Calculate correlation coefficient
    correlation = np.corrcoef(Y_val[:, i], Y_preds[:, i])[0, 1]
    
    # Add scatter plot for true vs. predicted
    fig.add_trace(go.Scatter(
        x=Y_val[:, i],  # True outputs for the i-th variable
        y=Y_preds[:, i],  # Predicted outputs for the i-th variable
        mode='markers',
        name=f'{output_name} (r={correlation:.2f})',
        marker=dict(size=6, opacity=0.7),
        showlegend=True
    ))

    # Add perfect correlation line (y=x)
    max_val = max(Y_val[:, i].max(), Y_preds[:, i].max())
    min_val = min(Y_val[:, i].min(), Y_preds[:, i].min())
    fig.add_trace(go.Scatter(
        x=[min_val, max_val],
        y=[min_val, max_val],
        mode='lines',
        name=f'{output_name} Perfect Correlation (y=x)',
        line=dict(color='black', dash='dash'),
        showlegend=False
    ))

# Update layout
fig.update_layout(
    title="True vs Predicted Outputs with Correlation Coefficients",
    xaxis_title="True Outputs",
    yaxis_title="Predicted Outputs",
    legend=dict(yanchor="top", y=0.3, xanchor="left", x=0.7),
    template="plotly_white"
)

# Show the plot
fig.show()


## Validate model with timeseries experimental data

In [46]:
idx_start = 10
idx_end = len(df)
df_mod = pd.DataFrame()

# Run model
for idx in range(idx_start,idx_end):
    ds = df.iloc[idx]
        
    logger.info(f"Iteration {idx} / {len(df)}")
    start_time = time.time()

    Tmed_c_out = ds["Tmed_c_out"]
    Tmed_c_in = ds["Tmed_c_in"]
    mmed_f = ds["qmed_f"] if abs(ds["qmed_f"]-5) > 0.5 else 5
    mmed_f = within_range_or_nan_or_max(mmed_f, range=(5, 9), var_name="qmed_f")
    mmed_s = within_range_or_zero_or_max(ds["qmed_s"], range=(30, 48), var_name="qmed_s")
    Tmed_s_in = within_range_or_nan_or_max(ds["Tmed_s_in"], range=(60, 75), var_name="Tmed_s_in")

    MsIn = matlab.double([mmed_s / 3.6], size=(1, 1))  # m³/h -> L/s
    TsinIn = matlab.double([Tmed_s_in], size=(1, 1))
    MfIn = matlab.double([mmed_f], size=(1, 1))
    TcwinIn = matlab.double([Tmed_c_in], size=(1, 1))
    op_timeIn = matlab.double([0], size=(1, 1))
                
    med_model_solved = False
    mmed_d = np.nan
    Tmed_s_out = np.nan
    mmed_c = np.nan
    while not med_model_solved and (Tmed_c_in < Tmed_c_out < 40) and mmed_f > 0 and mmed_s > 0 and Tmed_s_in > 0 and Tmed_s_in > 0:   
        # try:                     
        
    # if mmed_f > 0 and mmed_s > 0 and Tmed_s_in > 0 and Tmed_s_in > 0:
        TcwoutIn = matlab.double([Tmed_c_out], size=(1, 1))

        mmed_d, Tmed_s_out, mmed_c, _, _ = MED_model_matlab.MED_model(
            MsIn,  # L/s
            TsinIn,  # ºC
            MfIn,  # m³/h
            TcwoutIn,  # ºC
            TcwinIn,  # ºC
            op_timeIn,  # hours
            nargout=5
        )

        if mmed_c > 20:
            Tmed_c_out = Tmed_c_out + 1
        elif mmed_c < 9:
            Tmed_c_out = Tmed_c_out - 1
        else:
            med_model_solved = True
            
        # except Exception as e:
        #     # TODO: Put the right variable in the search and set the delta to the right value
        #     if re.search('mmed_c', str(e)):
        #         # self.penalty = self.default_penalty
        #         deltaTmed_cout = 0.5  # or -0.5
        #         logger.warning(f"Unfeasible operation in MED: {e}")
        #     else:
        #         raise e
        # 
        # # La dirección de cambio debería ser en función si el caudal de refrigeración es poco o demasiado
        # Tmed_c_out = np.min([Tmed_c_out + 0.5, 40])
        # Tmed_c_out = np.max([Tmed_c_out, ds["Tmed_c_in"]])
        # 
        # TcwoutIn = matlab.double([Tmed_c_out], size=(1, 1))
                  
    logger.info(f"Elapsed time: {time.time()-start_time:.2f} seconds")
    
    result = pd.DataFrame(
        {
            "qmed_f": mmed_f,
            "qmed_d": mmed_d,
            "qmed_c": mmed_c,
            "qmed_s": mmed_s,
            "Tmed_s_in": Tmed_s_in,
            "Tmed_s_out": Tmed_s_out,
            "Tmed_c_in": ds["Tmed_c_in"],
            "Tmed_c_out": Tmed_c_out,
        },
        index=[0]
    )
    
    df_mod = pd.concat([df_mod, result], ignore_index=True)


2025-01-04 08:35:08.203 | INFO     | __main__:<module>:9 - Iteration 10 / 89
2025-01-04 08:35:08.204 | INFO     | __main__:<module>:67 - Elapsed time: 0.00 seconds
2025-01-04 08:35:08.206 | INFO     | __main__:<module>:9 - Iteration 11 / 89
2025-01-04 08:35:08.206 | INFO     | __main__:<module>:67 - Elapsed time: 0.00 seconds
2025-01-04 08:35:08.208 | INFO     | __main__:<module>:9 - Iteration 12 / 89
2025-01-04 08:35:08.208 | INFO     | __main__:<module>:67 - Elapsed time: 0.00 seconds
2025-01-04 08:35:08.209 | INFO     | __main__:<module>:9 - Iteration 13 / 89
2025-01-04 08:35:08.210 | INFO     | __main__:<module>:67 - Elapsed time: 0.00 seconds
2025-01-04 08:35:08.211 | INFO     | __main__:<module>:9 - Iteration 14 / 89
2025-01-04 08:35:08.212 | INFO     | __main__:<module>:67 - Elapsed time: 0.00 seconds
2025-01-04 08:35:08.213 | INFO     | __main__:<module>:9 - Iteration 15 / 89
2025-01-04 08:35:08.214 | INFO     | __main__:<module>:67 - Elapsed time: 0.00 seconds
2025-01-04 08:35

2025-01-04 08:35:09.701 | INFO     | __main__:<module>:67 - Elapsed time: 1.48 seconds
2025-01-04 08:35:09.703 | INFO     | __main__:<module>:9 - Iteration 21 / 89
2025-01-04 08:35:09.762 | INFO     | __main__:<module>:67 - Elapsed time: 0.06 seconds
2025-01-04 08:35:09.764 | INFO     | __main__:<module>:9 - Iteration 22 / 89
2025-01-04 08:35:09.783 | INFO     | __main__:<module>:67 - Elapsed time: 0.02 seconds
2025-01-04 08:35:09.785 | INFO     | __main__:<module>:9 - Iteration 23 / 89
2025-01-04 08:35:09.799 | INFO     | __main__:<module>:67 - Elapsed time: 0.01 seconds
2025-01-04 08:35:09.801 | INFO     | __main__:<module>:9 - Iteration 24 / 89
2025-01-04 08:35:09.815 | INFO     | __main__:<module>:67 - Elapsed time: 0.01 seconds
2025-01-04 08:35:09.816 | INFO     | __main__:<module>:9 - Iteration 25 / 89
2025-01-04 08:35:09.831 | INFO     | __main__:<module>:67 - Elapsed time: 0.01 seconds
2025-01-04 08:35:09.833 | INFO     | __main__:<module>:9 - Iteration 26 / 89
2025-01-04 08:35

In [47]:
# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx < idx_end - 1 else idx_end]

with open(data_path / 'plt_config_med.json') as f:
    plt_config = json.load(f)

fig = experimental_results_plot(plt_config, df, df_comp=df_mod, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'med_validation_{df.index[0].strftime("%Y%m%d")}')
)
